In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math as mth
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
import seaborn as sns
import statsmodels.api as sm
from scipy import stats
from scipy.signal import argrelextrema
from sklearn.model_selection import train_test_split
from statsmodels.graphics.gofplots import qqplot
from scipy.stats import shapiro, normaltest
from sklearn.metrics import classification_report
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import scipy.stats as ss
%matplotlib inline

In [ ]:
train = pd.read_csv('D:\\kunal_DS\\train.csv')
train.head()

In [15]:
from datetime import datetime
def conv_dob_year(date_string, str_format="%d-%m-%y"):
    todays_d = datetime.now()
    date_t = datetime.strptime(date_string, str_format)
    delta = todays_d - date_t
    n_days=delta.days
    if(n_days<0):
        n_days = n_days + 100*365
    return n_days/365
def conv_y_m_to_years(y_m_string):
    tmp_1 = y_m_string.split("yrs")
    tmp_2=tmp_1[0].strip()
    tmp_3 = tmp_1[1].split("mon")[0].strip()
    return (int(tmp_2)*365+30*int(tmp_3))/365

In [16]:
train1=train.copy()

In [17]:
train1['Date.of.Birth'] = train1['Date.of.Birth'].apply(conv_dob_year)
train1['DisbursalDate'] = train1['DisbursalDate'].apply(conv_dob_year)
train1['Age_as_on_disbursal'] = train1['Date.of.Birth']-train1['DisbursalDate']
train1['AVERAGE.ACCT.AGE']=train1['AVERAGE.ACCT.AGE'].apply(conv_y_m_to_years)
train1['CREDIT.HISTORY.LENGTH']=train1['CREDIT.HISTORY.LENGTH'].apply(conv_y_m_to_years)

In [18]:
train1.head()

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,...,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES,loan_default,Age_as_on_disbursal
0,420825,50578,58400,89.55,67,22807,45,1441,35.361644,Salaried,...,0,0,0,0,0,0.000000,0.000000,0,0,34.610959
1,537409,47145,65550,73.23,67,22807,45,1502,33.780822,Self employed,...,0,1991,0,0,1,1.904110,1.904110,0,1,33.178082
2,417566,53278,61360,89.63,67,22807,45,1497,33.715068,Self employed,...,0,0,0,0,0,0.000000,0.000000,0,0,32.958904
3,624493,57513,66113,88.48,67,22807,45,1501,25.358904,Self employed,...,0,31,0,0,0,0.657534,1.246575,1,1,24.838356
4,539055,52378,60300,88.39,67,22807,45,1495,41.427397,Self employed,...,0,0,0,0,0,0.000000,0.000000,1,1,40.824658


In [21]:
train1.isnull().sum()

UniqueID                               0
disbursed_amount                       0
asset_cost                             0
ltv                                    0
branch_id                              0
supplier_id                            0
manufacturer_id                        0
Current_pincode_ID                     0
Date.of.Birth                          0
Employment.Type                        0
DisbursalDate                          0
State_ID                               0
Employee_code_ID                       0
MobileNo_Avl_Flag                      0
Aadhar_flag                            0
PAN_flag                               0
VoterID_flag                           0
Driving_flag                           0
Passport_flag                          0
PERFORM_CNS.SCORE                      0
PERFORM_CNS.SCORE.DESCRIPTION          0
PRI.NO.OF.ACCTS                        0
PRI.ACTIVE.ACCTS                       0
PRI.OVERDUE.ACCTS                      0
PRI.CURRENT.BALA

In [20]:
train1['Employment.Type'].fillna('Unemployed',inplace=True)
train1['Employment.Type'].value_counts()

Self employed    127635
Salaried          97858
Unemployed         7661
Name: Employment.Type, dtype: int64

In [22]:
train1.isnull().sum()

UniqueID                               0
disbursed_amount                       0
asset_cost                             0
ltv                                    0
branch_id                              0
supplier_id                            0
manufacturer_id                        0
Current_pincode_ID                     0
Date.of.Birth                          0
Employment.Type                        0
DisbursalDate                          0
State_ID                               0
Employee_code_ID                       0
MobileNo_Avl_Flag                      0
Aadhar_flag                            0
PAN_flag                               0
VoterID_flag                           0
Driving_flag                           0
Passport_flag                          0
PERFORM_CNS.SCORE                      0
PERFORM_CNS.SCORE.DESCRIPTION          0
PRI.NO.OF.ACCTS                        0
PRI.ACTIVE.ACCTS                       0
PRI.OVERDUE.ACCTS                      0
PRI.CURRENT.BALA

In [23]:
import scipy.stats as stats
from scipy.stats import chi2_contingency

class ChiSquare:
    def __init__(self, dataframe):
        self.df = dataframe
        self.p = None #P-Value
        self.chi2 = None #Chi Test Statistic
        self.dof = None
        
        self.dfObserved = None
        self.dfExpected = None
        
    def _print_chisquare_result(self, colX, alpha):
        result = ""
        if self.p<alpha:
            result="{0} is IMPORTANT for Prediction".format(colX)
        else:
            result="{0} is NOT an important predictor. (Discard {0} from model)".format(colX)

        print(result)
        
    def TestIndependence(self,colX,colY, alpha=0.05):
        X = self.df[colX].astype(str)
        Y = self.df[colY].astype(str)
        
        self.dfObserved = pd.crosstab(Y,X) 
        chi2, p, dof, expected = stats.chi2_contingency(self.dfObserved.values)
        self.p = p
        self.chi2 = chi2
        self.dof = dof 
        
        self.dfExpected = pd.DataFrame(expected, columns=self.dfObserved.columns, index = self.dfObserved.index)
        
        self._print_chisquare_result(colX,alpha)

In [24]:
X = train1.drop(['UniqueID', 'loan_default'], axis=1)
ct = ChiSquare(train1)
for c in X.columns:
    ct.TestIndependence(c, 'loan_default')

disbursed_amount is IMPORTANT for Prediction
asset_cost is IMPORTANT for Prediction
ltv is IMPORTANT for Prediction
branch_id is IMPORTANT for Prediction
supplier_id is IMPORTANT for Prediction
manufacturer_id is IMPORTANT for Prediction
Current_pincode_ID is IMPORTANT for Prediction
Date.of.Birth is NOT an important predictor. (Discard Date.of.Birth from model)
Employment.Type is IMPORTANT for Prediction
DisbursalDate is IMPORTANT for Prediction
State_ID is IMPORTANT for Prediction
Employee_code_ID is IMPORTANT for Prediction
MobileNo_Avl_Flag is NOT an important predictor. (Discard MobileNo_Avl_Flag from model)
Aadhar_flag is IMPORTANT for Prediction
PAN_flag is NOT an important predictor. (Discard PAN_flag from model)
VoterID_flag is IMPORTANT for Prediction
Driving_flag is IMPORTANT for Prediction
Passport_flag is IMPORTANT for Prediction
PERFORM_CNS.SCORE is IMPORTANT for Prediction
PERFORM_CNS.SCORE.DESCRIPTION is IMPORTANT for Prediction
PRI.NO.OF.ACCTS is IMPORTANT for Predicti

In [27]:
train1.columns

Index(['UniqueID', 'disbursed_amount', 'asset_cost', 'ltv', 'branch_id',
       'supplier_id', 'manufacturer_id', 'Current_pincode_ID', 'Date.of.Birth',
       'Employment.Type', 'DisbursalDate', 'State_ID', 'Employee_code_ID',
       'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE',
       'PERFORM_CNS.SCORE.DESCRIPTION', 'PRI.NO.OF.ACCTS', 'PRI.ACTIVE.ACCTS',
       'PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE', 'PRI.SANCTIONED.AMOUNT',
       'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS', 'SEC.ACTIVE.ACCTS',
       'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE', 'SEC.SANCTIONED.AMOUNT',
       'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT', 'SEC.INSTAL.AMT',
       'NEW.ACCTS.IN.LAST.SIX.MONTHS', 'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS',
       'AVERAGE.ACCT.AGE', 'CREDIT.HISTORY.LENGTH', 'NO.OF_INQUIRIES',
       'loan_default', 'Age_as_on_disbursal'],
      dtype='object')

In [34]:
train1=train1.drop(columns=['Date.of.Birth','MobileNo_Avl_Flag','PAN_flag','PRI.CURRENT.BALANCE','PRI.SANCTIONED.AMOUNT','PRI.DISBURSED.AMOUNT','SEC.NO.OF.ACCTS','SEC.OVERDUE.ACCTS','SEC.CURRENT.BALANCE','SEC.SANCTIONED.AMOUNT','SEC.DISBURSED.AMOUNT','PRIMARY.INSTAL.AMT','SEC.INSTAL.AMT','Age_as_on_disbursal'])

In [36]:
train1.columns

Index(['UniqueID', 'disbursed_amount', 'asset_cost', 'ltv', 'branch_id',
       'supplier_id', 'manufacturer_id', 'Current_pincode_ID',
       'Employment.Type', 'DisbursalDate', 'State_ID', 'Employee_code_ID',
       'Aadhar_flag', 'VoterID_flag', 'Driving_flag', 'Passport_flag',
       'PERFORM_CNS.SCORE', 'PERFORM_CNS.SCORE.DESCRIPTION', 'PRI.NO.OF.ACCTS',
       'PRI.ACTIVE.ACCTS', 'PRI.OVERDUE.ACCTS', 'SEC.ACTIVE.ACCTS',
       'NEW.ACCTS.IN.LAST.SIX.MONTHS', 'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS',
       'AVERAGE.ACCT.AGE', 'CREDIT.HISTORY.LENGTH', 'NO.OF_INQUIRIES',
       'loan_default'],
      dtype='object')

In [41]:
train1=train1.drop(columns=['UniqueID'])

In [42]:
train1.columns

Index(['disbursed_amount', 'asset_cost', 'ltv', 'branch_id', 'supplier_id',
       'manufacturer_id', 'Current_pincode_ID', 'Employment.Type',
       'DisbursalDate', 'State_ID', 'Employee_code_ID', 'Aadhar_flag',
       'VoterID_flag', 'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE',
       'PERFORM_CNS.SCORE.DESCRIPTION', 'PRI.NO.OF.ACCTS', 'PRI.ACTIVE.ACCTS',
       'PRI.OVERDUE.ACCTS', 'SEC.ACTIVE.ACCTS', 'NEW.ACCTS.IN.LAST.SIX.MONTHS',
       'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'AVERAGE.ACCT.AGE',
       'CREDIT.HISTORY.LENGTH', 'NO.OF_INQUIRIES', 'loan_default'],
      dtype='object')

In [43]:
train1.head()

,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Employment.Type,DisbursalDate,State_ID,...,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,SEC.ACTIVE.ACCTS,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES,loan_default
0,50578,58400,89.55,67,22807,45,1441,Salaried,0.750685,6,...,0,0,0,0,0,0,0.000000,0.000000,0,0
1,47145,65550,73.23,67,22807,45,1502,Self employed,0.602740,6,...,1,1,1,0,0,1,1.904110,1.904110,0,1
2,53278,61360,89.63,67,22807,45,1497,Self employed,0.756164,6,...,0,0,0,0,0,0,0.000000,0.000000,0,0
3,57513,66113,88.48,67,22807,45,1501,Self employed,0.520548,6,...,3,0,0,0,0,0,0.657534,1.246575,1,1
4,52378,60300,88.39,67,22807,45,1495,Self employed,0.602740,6,...,0,0,0,0,0,0,0.000000,0.000000,1,1


In [44]:
x=train1.drop(columns=['loan_default'])
y=train1['loan_default']

In [45]:
dummy = pd.get_dummies(x,columns=['Employment.Type','PERFORM_CNS.SCORE.DESCRIPTION'],drop_first = True)

In [46]:
lr= LogisticRegression()

In [47]:
x_train,x_test,y_train,y_test=train_test_split(dummy,y,train_size=0.7,random_state=7)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [48]:
model_fit= lr.fit(x_train,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [49]:
y_pred = lr.predict(x_test)

In [50]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.78      1.00      0.88     54695
           1       0.17      0.00      0.00     15252

   micro avg       0.78      0.78      0.78     69947
   macro avg       0.48      0.50      0.44     69947
weighted avg       0.65      0.78      0.69     69947



In [56]:
rfc=RandomForestClassifier()
X_train_rfc,X_test_rfc,Y_train_rfc,Y_test_rfc=train_test_split(dummy,y,test_size=0.30,random_state=7)

In [57]:

rfc.fit(X_train_rfc,Y_train_rfc)
Y_pred_rfc=rfc.predict(X_test_rfc)
print(classification_report(Y_test_rfc,Y_pred_rfc))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


              precision    recall  f1-score   support

           0       0.79      0.96      0.87     54695
           1       0.36      0.08      0.13     15252

   micro avg       0.77      0.77      0.77     69947
   macro avg       0.57      0.52      0.50     69947
weighted avg       0.69      0.77      0.71     69947



In [58]:
dtc=DecisionTreeClassifier()
X_train_dtc,X_test_dtc,Y_train_dtc,Y_test_dtc=train_test_split(dummy,y,test_size=0.30,random_state=7)

dtc.fit(X_train_dtc,Y_train_dtc)
Y_pred_dtc=dtc.predict(X_test_dtc)

print(classification_report(Y_test_dtc,Y_pred_dtc))

              precision    recall  f1-score   support

           0       0.79      0.78      0.79     54695
           1       0.26      0.28      0.27     15252

   micro avg       0.67      0.67      0.67     69947
   macro avg       0.53      0.53      0.53     69947
weighted avg       0.68      0.67      0.67     69947



In [59]:
from sklearn.feature_selection import RFE
lr = LogisticRegression()
rfe = RFE(estimator=lr, n_features_to_select=10, verbose=3)
rfe.fit(dummy, y)
rfe_df1 = rfe.fit_transform(dummy, y)

Fitting estimator with 45 features.


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Fitting estimator with 44 features.


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Fitting estimator with 43 features.


...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Fitting estimator with 12 features.


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Fitting estimator with 11 features.


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [61]:
print("Features sorted by their rank:")
print(sorted(zip(map(lambda x: round(x, 4), rfe.ranking_), X.columns)))

Features sorted by their rank:
[(1, 'CREDIT.HISTORY.LENGTH'), (1, 'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS'), (1, 'Date.of.Birth'), (1, 'Employee_code_ID'), (1, 'NEW.ACCTS.IN.LAST.SIX.MONTHS'), (1, 'SEC.ACTIVE.ACCTS'), (1, 'SEC.INSTAL.AMT'), (1, 'SEC.NO.OF.ACCTS'), (1, 'SEC.OVERDUE.ACCTS'), (2, 'PRI.CURRENT.BALANCE'), (3, 'PRIMARY.INSTAL.AMT'), (4, 'PRI.NO.OF.ACCTS'), (5, 'Driving_flag'), (6, 'PRI.SANCTIONED.AMOUNT'), (7, 'State_ID'), (8, 'Passport_flag'), (9, 'PRI.OVERDUE.ACCTS'), (10, 'NO.OF_INQUIRIES'), (11, 'PRI.ACTIVE.ACCTS'), (12, 'PERFORM_CNS.SCORE.DESCRIPTION'), (13, 'ltv'), (14, 'Employment.Type'), (15, 'PERFORM_CNS.SCORE'), (16, 'VoterID_flag'), (18, 'manufacturer_id'), (19, 'branch_id'), (20, 'PAN_flag'), (21, 'disbursed_amount'), (22, 'asset_cost'), (23, 'SEC.CURRENT.BALANCE'), (24, 'PRI.DISBURSED.AMOUNT'), (26, 'MobileNo_Avl_Flag'), (27, 'Current_pincode_ID'), (28, 'DisbursalDate'), (29, 'AVERAGE.ACCT.AGE'), (30, 'SEC.SANCTIONED.AMOUNT'), (32, 'Aadhar_flag'), (34, 'supplier_id

In [62]:
  rfe_df1

array([[0.75068493, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.60273973, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.75616438, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.5260274 , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.59452055, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.51780822, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

# Logistic regression model after performing RFE

In [78]:
lr= LogisticRegression()
x_train,x_test,y_train,y_test=train_test_split(rfe_df1,y,train_size=0.7,random_state=7)
model_fit= lr.fit(x_train,y_train)
y_pred = lr.predict(x_test)
print(classification_report(y_test,y_pred))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

           0       0.78      1.00      0.88     54695
           1       0.00      0.00      0.00     15252

   micro avg       0.78      0.78      0.78     69947
   macro avg       0.39      0.50      0.44     69947
weighted avg       0.61      0.78      0.69     69947



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


#  RandomForestClassifier model after performing RFE

In [68]:
rfc=RandomForestClassifier()
X_train_rfc,X_test_rfc,Y_train_rfc,Y_test_rfc=train_test_split(rfe_df1,y,test_size=0.30,random_state=7)
rfc.fit(X_train_rfc,Y_train_rfc)
Y_pred_rfc=rfc.predict(X_test_rfc)
print(classification_report(Y_test_rfc,Y_pred_rfc))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


              precision    recall  f1-score   support

           0       0.78      1.00      0.88     54695
           1       0.31      0.00      0.01     15252

   micro avg       0.78      0.78      0.78     69947
   macro avg       0.55      0.50      0.44     69947
weighted avg       0.68      0.78      0.69     69947



# DecisionTree model after performing RFE

In [69]:
dtc=DecisionTreeClassifier()
X_train_dtc,X_test_dtc,Y_train_dtc,Y_test_dtc=train_test_split(rfe_df1,y,test_size=0.30,random_state=7)

dtc.fit(X_train_dtc,Y_train_dtc)
Y_pred_dtc=dtc.predict(X_test_dtc)

print(classification_report(Y_test_dtc,Y_pred_dtc))

              precision    recall  f1-score   support

           0       0.78      1.00      0.88     54695
           1       0.31      0.00      0.01     15252

   micro avg       0.78      0.78      0.78     69947
   macro avg       0.55      0.50      0.44     69947
weighted avg       0.68      0.78      0.69     69947



In [70]:
pip install imblearn


The following command must be run outside of the IPython shell:

    $ pip install imblearn

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [82]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=7)
X_ov, y_ov = smote.fit_resample(rfe_df1, y)
X_train, X_test, y_train, y_test = train_test_split(X_ov, y_ov, train_size=0.7, random_state=7)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [88]:
rfc.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [89]:
y_pred=rfc.predict(X_test)

In [90]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.56      0.56      0.56     54812
           1       0.56      0.56      0.56     54714

   micro avg       0.56      0.56      0.56    109526
   macro avg       0.56      0.56      0.56    109526
weighted avg       0.56      0.56      0.56    109526



In [ ]:
rf.fit(dummy.drop(['UniqueID', 'loan_default', 'PERFORM_CNS.SCORE.DESCRIPTION'], axis=1), dummy['loan_default'])
pd.Series(rf.feature_importances_, index=dummy.drop(['UniqueID', 'loan_default', 'PERFORM_CNS.SCORE.DESCRIPTION'], axis=1).columns).plot.bar(color='steelblue', figsize=(12, 6))

from sklearn.feature_selection import SelectFromModel
trans = SelectFromModel(rf, threshold=0.01)
X_fe_model = trans.fit_transform(loan_dummified.drop(['UniqueID', 'loan_default', 'PERFORM_CNS.SCORE.DESCRIPTION'], axis=1), loan_dummified['loan_default'])

X_ov3, y_ov3 = smote.fit_sample(X_fe_model, loan_dummified['loan_default'])

X_train, X_test, y_train, y_test = train_test_split(X_ov3, y_ov3, train_size=0.7, random_state=7)

lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print(classification_report(y_test, y_pred))

rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred))